# 파이썬 프로젝트 빠르게 한 것의 코드 일부

In [ ]:
# v2-2-1
# - 추후 하고 싶으면 할 일 (1) 언어 추가 (2) 성능 업그레이드 (3) 추론 속도 증가를 위한 시간 조정 : 빠른 방법이라 사용한 초 제한 말고도 진행

# - 참고
#   + [1] 허깅페이스 번역 코드 https://huggingface.co/openai/whisper-small
#     > [2] 파이프라인 :  https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.AutomaticSpeechRecognitionPipeline
#   + [3] 앞처리 코드 : 출처 묻기
#   + [4] 그라디오 실시간 STT : https://www.gradio.app/guides/real-time-speech-recognition

#   + > [5] 위스퍼 공식 및 언어 정보 (오픈소스) : https://github.com/openai/whisper
#   + [6] 영상 음성 : https://mep997.tistory.com/42
#   + [7] 더 많은 자료 및 실험은 원본 주피터 노트북, 문서 확인

# - [8] 위스퍼 함수 https://huggingface.co/docs/transformers/v4.23.0/en/model_doc/whisper
# - 트랜스포머에서 함수 소스 찾음 : [9] https://github.com/huggingface/transformers/blob/v4.23.0/src/transformers/models/whisper/processing_whisper.py#L22

import gradio as gr
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa
import numpy as np
import torch
from transformers import pipeline
device = "cuda:0" if torch.cuda.is_available() else "cpu" #~
print(device)


# 추론 속도를 위해 함수 밖으로 빼준 부분 & 코드 실험 시간을 위해 셀 분리
# [`WhisperProcessor`] offers all the functionalities of [`WhisperFeatureExtractor`] and [`WhisperTokenizer`]. [5][9]
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.to(device)
# 트랜스포머에서 함수 소스 찾음 : [9] https://github.com/huggingface/transformers/blob/v4.23.0/src/transformers/models/whisper/processing_whisper.py#L22
forced_decoder_ids = torch.tensor(processor.get_decoder_prompt_ids(language="english", task="translate"), dtype=torch.long).to(device)#processor.get_decoder_prompt_ids(language="english", task="translate").to(device)

# [1][2]
# pipe = pipeline(
#   "automatic-speech-recognition",
#   model="openai/whisper-small",
#   device=device)


In [ ]:
gr.close_all()
def translate_speech(audio_file): # .wav파일
    pass # 다른 사람 코드도 조금 들어가 있는데 출처 달기 애매해서 지움

# Either 'wav' or 'mp3'. wav files are lossless but will tend to be larger files. mp3 files tend to be smaller. [https://www.gradio.app]
interface = gr.Interface(
    fn=translate_speech,
    inputs=gr.Audio(sources=["upload", "microphone"], format=['wav','mp3'], type="filepath", label="Record your speech"),
    outputs=gr.Textbox(label="Translated Text"),
    title="Whisper Translation",
    description="Put the Record button. It will translate voices from various languages ​​into English in real time.")


# 최종 사용하진 않은 방법 : 모델 파이프라인으로 가져오기
# m1 그라디오 :  transcriber   = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")
# m2 허깅페이스 - 위스퍼
# pipe = pipeline(
#   "automatic-speech-recognition",
#   model="openai/whisper-small",
#   chunk_length_s=30,
#   device=device,
# )

def live_translate(stream, new_chunk):
    # 그라디오
    orig_sr, y = new_chunk # gradio가 주는 sample rate
    sr=16000

    y = y.astype(np.float32) # 위스퍼 관련 함수 입력 형식을 위해 or 정규화를 위해 int16이면 안됨
    y /= np.max(np.abs(y)) #  정규화 -1~1
    y = librosa.resample(y, orig_sr=orig_sr, target_sr=sr)

    # k초 동안의 샘플 수
    num_samples_for_8_sec = int(8 * sr)  # k초 * sample rate
    if stream is not None:
        stream = np.concatenate( (stream, y))
        if len(stream) > num_samples_for_8_sec:
            y = stream[-num_samples_for_8_sec:] 
        else:
            y=stream
    else:
        stream =y
    input= np.expand_dims(y, axis=0) #stream에는 추가할 필요없이 보존 데이터일 뿐임
    # temp3.append('----1-----')

    input_features = processor(input, sampling_rate=sr, return_tensors="pt").input_features.to(device)

    # - 모델 입출력
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    return stream, transcription[0]
    # return stream, pipe(raw=stream, sampling_rate=sr, generate_kwargs = {"task":"translate", "language":"<|en|>"} )['text']
    # 언어관련 https://github.com/openai/whisper/blob/main/whisper/tokenizer.py
    # return stream, pipe({"sampling_rate": sr, "raw": stream})["text"]
live_streaming= gr.Interface(
    fn=live_translate,
    inputs=["state", gr.Audio(source="microphone", label="Record your speech",streaming=True)],
    outputs=["state", gr.Textbox(label="Translated Text"), ],
    title="Real-time Voice Translation",
    description="Put the Record button. It will translate voices from various languages ​​into English in real time.",
    live=True
)


demo = gr.TabbedInterface([live_streaming, interface], ["Real-time Voice Translation", "Slow Translation"])
demo.launch()
